# Extra - introduction au scraping

## Du pixel aux images - 32M7138

*Printemps 2025 - Université de Genève*

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

## **Plan du cours**

> **Récupération automatique des données (scraping)**
> * Les librairies nécessaire
> * Chercher les données dans une page HTML d'après une URL
> * Lister toutes les pages
> * Automatiser la récupération 
> * Sauvegarder au format CSV

### Les librairies nécessaire

In [ ]:
# importer librairies

import requests
from bs4 import BeautifulSoup
import pandas as pd

### Chercher les données dans une page HTML d'après une URL

In [ ]:
# Lien que vous souhaitez télécharger
url = "https://archive.org/details/crash-magazine-01"

# Télécharger le contenu HTML
response = requests.get(url)
print(response)

In [ ]:
# Vérifier si la requête a réussi (code 200)
if response.status_code == 200:
    # Récupérer le contenu HTML
    html_content = response.text
    print(html_content)
else:
    print(f"Erreur {response.status_code} lors du téléchargement du lien.")

In [ ]:
# date de publication

<dl class="metadata-definition">
        <dt>Publication date</dt>
        <dd class="">
          <a href="/search.php?query=date:1984-02">
            <span itemprop="datePublished">1984-02</span>
        </a>
                </dd>
    </dl>

In [ ]:
# Utiliser BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extraire le titre de la page
title = soup.title.text if soup.title else "Titre non trouvé"

# Extraire la date de publication
publication_date_tag = soup.find("dt", text="Publication date")
publication_date = publication_date_tag.find_next("span", {"itemprop": "datePublished"}).text if publication_date_tag else "Date non trouvée"
print(publication_date)

In [ ]:
# balise pour le pdf

<a class="format-summary download-pill" href="/download/crash-magazine-01/Crash_01_Feb_1984.pdf" title="" data-toggle="tooltip" data-placement="auto left" data-container="body" data-original-title="31.8M">
                PDF                <span class="iconochive-download" aria-hidden="true"></span><span class="icon-label sr-only">download</span>              </a>

In [ ]:
# lien pour le pdf
soup = BeautifulSoup(html_content, 'html.parser')

# Extraire le lien vers le fichier PDF
pdf_link_tag = soup.find("a", {"class": "stealth","title": "31.8M", "href": lambda href: href and "download" in href})
pdf_link = pdf_link_tag["href"] if pdf_link_tag else "Lien PDF non trouvé"

print(pdf_link)

In [ ]:
# récupérer le titre

# Extraire l'identificateur
identifier_tag = soup.find("dt", text="Identifier")
identifier = identifier_tag.find_next("span", {"itemprop": "identifier"}).text if identifier_tag else "Identificateur non trouvé"
print(identifier)

In [ ]:
# stocker les informations dans un tableur structuré

df = pd.DataFrame({"Titre": [identifier],
                       "Date_publication": [publication_date],
                       "Lien_PDF": [pdf_link]})
df

### Lister toutes les pages

In [ ]:
# Générer la liste des URLs de 01 à 100
base_url = "https://archive.org/details/crash-magazine-{}"
urls = [base_url.format(str(i).zfill(2)) for i in range(1, 21)]

# Vérifier les URLs qui retournent un code 200
valid_urls = []

for url in urls:
    response = requests.head(url)  # Utilisation de HEAD pour économiser la bande passante
    if response.status_code == 200:
        valid_urls.append(url)
    else:
        continue

print(len(valid_urls))
print(urls[:2])  # Affiche les 5 premières URLs
print(urls[-2:]) # Affiche les 5 dernières URLs

### Automatiser la récupération 

In [ ]:
# Liste pour stocker les résultats
data_list = []

# Parcourir chaque URL et extraire les informations
for url in urls[:3]:
    response = requests.get(url)

    if response.status_code == 200:
        print(f"Extraction des données depuis {url}")

        # Analyser le HTML avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraire le titre de la page
        title = soup.title.text.strip() if soup.title else "Titre non trouvé"
        title = title.split(":")[0]

        # Extraire la date de publication
        publication_date_tag = soup.find("dt", text="Publication date")
        publication_date = publication_date_tag.find_next("span", {"itemprop": "datePublished"}).text.strip() if publication_date_tag else "Date non trouvée"

        # Extraire l'identificateur
        identifier_tag = soup.find("dt", text="Identifier")
        identifier = identifier_tag.find_next("span", {"itemprop": "identifier"}).text.strip() if identifier_tag else "Identificateur non trouvé"

        # Extraire le lien du PDF
        #pdf_link_tag = soup.find("a", {"class": "stealth", "href": lambda href: href and "download" in href})
        #pdf_link = "https://archive.org" + pdf_link_tag["href"] if pdf_link_tag else "Lien PDF non trouvé"

        pdf_link_tag = soup.find("a", {"class": "stealth"}, href=lambda href: href and href.endswith(".pdf"))
    
        if pdf_link_tag:
            pdf_link = pdf_link_tag["href"]
            # Compléter l'URL si elle est relative
            if pdf_link.startswith("/"):
                pdf_link = "https://archive.org" + pdf_link
        else:
            pdf_link = "Lien PDF non trouvé"
    
        
        # Stocker les données
        data_list.append({
            "URL": url,
            "Titre": title,
            "Date de publication": publication_date,
            "Identificateur": identifier,
            "Lien PDF": pdf_link
        })

    else:
        print(f"{url} inaccessible (code {response.status_code})")

# Affichage des résultats
print("\n Résultats :")
for data in data_list:
    print(data)

### Sauvegarder au format CSV

In [ ]:
df = pd.DataFrame(data_list, columns=["URL", "Titre", "Date de publication", "Identificateur", "Lien PDF"])
df

In [ ]:
df.to_csv("crash_magazines.csv", index=False, encoding="utf-8")

In [ ]:
# télécharger les pdfs

import os


# Création du dossier pour stocker les PDFs
pdf_folder = "pdfs"
os.makedirs(pdf_folder, exist_ok=True)

# Parcourir chaque ligne du DataFrame
for index, row in df.iterrows():
    pdf_link = row["Lien PDF"]
    identifier = row["Identificateur"]
    pdf_filename = f"{pdf_folder}/{identifier}.pdf"

    # Vérifier si le lien PDF est valide
    if pdf_link != "Lien PDF non trouvé" and isinstance(pdf_link, str) and pdf_link.startswith("http"):
        print(f"Téléchargement de {pdf_filename}...")

        # Télécharger le PDF
        pdf_response = requests.get(pdf_link, stream=True)
        if pdf_response.status_code == 200:
            with open(pdf_filename, "wb") as pdf_file:
                for chunk in pdf_response.iter_content(1024):
                    pdf_file.write(chunk)
            print(f"PDF enregistré : {pdf_filename}")
        else:
            print(f"Erreur lors du téléchargement du PDF pour {identifier}")
    else:
        print(f"⚠ Aucun lien PDF valide pour {identifier}")

print("\nTéléchargement terminé. Tous les PDFs sont dans le dossier 'pdfs/'.")
